# transforms tutorial

## screen size? 305dpi?

In [ ]:
def getAxesSizePx(fig, ax):
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    width, height = int(bbox.width*fig.dpi), int(bbox.height*fig.dpi)
    return width, height


matplotlib.rcParams['figure.dpi'] = 305

In [ ]:
import numpy as np
np.sqrt(3400**2+1800**2)/13.

In [ ]:
fig, ax = plt.subplots()
ax.text(0.5,0.5,'hello')

In [ ]:
fig.dpi

## data <-> display (screen) : coordinates transforms

In [ ]:
%matplotlib widget
from matplotlib import cm, pyplot as plt
import numpy as np

x = np.arange(0, 10, 0.005)
y = np.exp(-x/2.) * np.sin(2*np.pi*x)

fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlim(0, 10)
ax.set_ylim(-1, 1)

xdata, ydata = 5, 0
xdisplay, ydisplay = ax.transData.transform((xdata, ydata))
# ax.transData instance to transform from your data to your display

inv = ax.transData.inverted()
xdataInv,ydataInv = inv.transform((xdisplay, ydisplay))
# ax.transData.inverted() instance to transform from your display to your data

assert (np.round([xdataInv,ydataInv],15)==np.round([xdata,ydata],15)).all() , 'All is bad'

bbox = dict(boxstyle="round", fc="0.8")
arrowprops = dict(
    arrowstyle="->",
    connectionstyle="angle,angleA=0,angleB=90,rad=10")

offset = 72
ax.annotate('data = (%.1f, %.1f)' % (xdata, ydata),
            (xdata, ydata), xytext=(-2*offset, offset), textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)

disp = ax.annotate('display = (%.1f, %.1f)' % (xdisplay, ydisplay),
               #(xdisplay, ydisplay), xytext=(0.5*offset, -offset),
                (xdata, ydata), xytext=(0.5*offset, -offset),   
               #xycoords='figure pixels',
                xycoords=ax.transData,
               textcoords='offset points',
               bbox=bbox, arrowprops=arrowprops)

def onresize(event):
    xdisplay, ydisplay = ax.transData.transform((xdata, ydata))
    disp._text='Re_display = (%.1f, %.1f)' % (xdisplay, ydisplay)
    disp.set_text='Not working'
    disp.text='Working less'
    # .text or .set_text doesn't do anything!

cid = fig.canvas.mpl_connect('resize_event', onresize)

plt.show()

# Axes

In [ ]:
fig = plt.figure()
for i, label in enumerate(('A', 'B', 'C', 'D')):
    ax = fig.add_subplot(2, 2, i+1)
    #ax.text(0.05, 0.95, label, transform=ax.transAxes,
    ax.text(0.25+i/10, 0.5+i/10, label, transform=fig.transFigure, #ax.transAxes
            fontsize=16, fontweight='bold', va='top')

plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

fig, (ax1,ax2) = plt.subplots(1,2)
x, y = 10*np.random.rand(2, 1000)
ax1.plot(x, y, 'go', alpha=0.2)  # plot some data in data coordinates

circ = Circle((0.5, 0.5), 0.25, transform=fig.transFigure,#ax2.transAxes,#
                       facecolor='blue', alpha=0.75)
ax2.add_patch(circ)
plt.show()

In [ ]:
fig.canvas.draw()

In [ ]:
fig.clf()

In [ ]:
plt.close()

## matplotlib/examples/images_contours_and_fields/affine_image.py

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms


def get_image():
    delta = 0.25
    x = y = np.arange(-3.0, 3.0, delta)
    X, Y = np.meshgrid(x, y)
    Z1 = np.exp(-X**2 - Y**2)
    Z2 = np.exp(-(X - 1)**2 - (Y - 1)**2)
    Z = (Z1 - Z2)
    return Z


def do_plot(ax, Z, transform):
    im = ax.imshow(Z, interpolation='none',
                   origin='lower',
                   extent=[-2, 4, -3, 2], clip_on=True)

    trans_data = transform + ax.transData
    im.set_transform(trans_data)

    # display intended extent of the image
    x1, x2, y1, y2 = im.get_extent()
    ax.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], "y--",
            transform=trans_data)
    ax.set_xlim(-5, 5)
    ax.set_ylim(-4, 4)


# prepare image and figure
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
Z = get_image()

# image rotation
do_plot(ax1, Z, mtransforms.Affine2D().rotate_deg(30))

# image skew
do_plot(ax2, Z, mtransforms.Affine2D().skew_deg(30, 15))

# scale and reflection
do_plot(ax3, Z, mtransforms.Affine2D().scale(-1, .5))

# everything and a translation
do_plot(ax4, Z, mtransforms.Affine2D().
        rotate_deg(30).skew_deg(30, 15).scale(-1, .5).translate(.5, -1))

plt.show()

Canvas(header_visible=False, layout=Layout(grid_area='center', min_height='400px'), toolbar=Toolbar(toolitems=…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## without rotation

In [14]:
import numpy as np
import matplotlib.pyplot as plt


def get_image():
    delta = 0.25
    x = y = np.arange(-3.0, 3.0, delta)
    X, Y = np.meshgrid(x, y)
    Z1 = np.exp(-X**2 - Y**2)
    Z2 = np.exp(-(X - 1)**2 - (Y - 1)**2)
    Z = (Z1 - Z2)
    return Z


def do_plot(ax, Z):
    im = ax.imshow(Z, interpolation='none',
                   origin='lower',
                   extent=[-2, 4, -3, 2], clip_on=True)

    # display intended extent of the image
    x1, x2, y1, y2 = im.get_extent()
    ax.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1] )
    ax.set_xlim(-5, 5)
    ax.set_ylim(-4, 4)


# prepare image and figure

fig = plt.figure()
ax = fig.add_subplot(111)
Z = get_image()

do_plot(ax, Z)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …